# Manejo de datos depositados en una Base de datos 

## 2. Conceptos clave: BioProject, BioSample, SRA y GEO
De manera general, un BioProject (PRJNAxxxxxx) describe el proyecto de investigación en su conjunto (por ejemplo, un estudio de RNA-Seq) y se vincula a una o varias BioSamples (SAMNxxxxxx), que documentan las características biológicas de cada muestra. A partir de estas BioSamples se generan los datos crudos de secuenciación, almacenados en el Sequence Read Archive (SRA, SRRxxxxxx).

En algunos casos, el proyecto también deposita información en Gene Expression Omnibus (GEO):

Cada GSE corresponde a un experimento completo.
Cada GSM corresponde a una muestra individual, asociada a sus archivos de secuenciación.
En resumen, la ruta BioProject → BioSample → SRA está siempre presente, mientras que la rama hacia GEO es opcional y depende del tipo de estudio. 



## Entrez 
Es un modulo de **biopython**  que permite efecturar descarga de base de datos  

### Resumen de funciones de Bio.Entrez
- `esearch`: Buscar IDs en una base de datos.
- `efetch`: Descargar información completa (texto, XML, secuencias, metadatos).
- `esummary`: Obtener un resumen de un ID (título, organismo, etc.). 
> La función esummary devuelve un diccionario que resume la información esencial de un BioProject, incluyendo datos sobre el organismo, tipo y objetivo del > > proyecto, fecha de registro, estado de secuenciación y más. La siguiente estructura muestra los campos más relevantes obtenidos para el proyecto
- `elink`: Encontrar vínculos entre bases de datos (ej. GSE → SRA).
- `egquery`: Buscar un término en todas las bases del NCBI y ver cuántos resultados hay.
- `espell`: Corregir errores ortográficos en la búsqueda.
- `einfo`: Ver información de una base de datos (campos, descripción, qué contiene).
- `read`: convierte la respuesta de NCBI, en un objeto de python(listas, diccionarios, strings, etc.).

In [ ]:
from Bio import Entrez 
import pandas as pd 
from io import StringIO 
import subprocess
import os 

Entrez.email= "ismadls@lcg.unam.mx"

proyec_acc= "PRJNA552284" 

#Hacemos la consulta con esaearch de el id en la base bioproject en especifico 
handle= Entrez.esearch(db="bioproject", term=proyec_acc)
#guardamos 
search= Entrez.read(handle) 
handle.close()

print(search)

#salida de la consulta 
"""
Count	Número total de resultados encontrados para la búsqueda.
RetMax	Máximo de resultados devueltos en la consulta actual (por defecto 20 si no se especifica).
RetStart	Índice del primer resultado devuelto (sirve para paginar los resultados).
IdList	Lista de identificadores únicos (UIDs) de los resultados encontrados. Estos UIDs se usan en efetch, esummary, etc.
TranslationSet	Conjunto de traducciones de términos de búsqueda que hace NCBI (ej. sinónimos o equivalencias). Puede estar vacío.
TranslationStack	Muestra paso a paso cómo NCBI interpretó tu búsqueda (ej. en qué campos buscó y cuántos resultados encontró).
QueryTranslation	Versión final de la consulta después de ser interpretada por NCBI (puede ser igual a tu búsqueda original o una forma ajustada).

""" 
#consulatmos el UID 
project_uid= search["IdList"][0] 

print(project_uid)


{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['552284'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'PRJNA552284[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'PRJNA552284[All Fields]'}
552284
{'DocumentSummarySet': DictElement({'DocumentSummary': [DictElement({'TaxId': '0', 'Project_Id': '552284', 'Project_Acc': 'PRJNA552284', 'Project_Type': 'Primary submission', 'Project_Data_Type': 'Raw sequence reads', 'Sort_By_ProjectType': '563532', 'Sort_By_DataType': '551612', 'Sort_By_Organism': '884981', 'Project_Subtype': '', 'Project_Target_Scope': 'Multispecies', 'Project_Target_Material': 'Genome', 'Project_Target_Capture': 'Whole', 'Project_MethodType': 'Sequencing', 'Project_Method': '', 'Project_Objectives_List': [{'Project_ObjectivesType': 'Raw Sequence Reads', 'Project_Objectives': ''}], 'Registration_Date': '2019/07/02 00:00', 'Project_Name': '', 'Project_Title': 'Babesia divergens strain:Bd Rouen 1987 | culti

### Consultar el resumen de un proyecto 

In [4]:
#consultamos el resumen del proyecto 

handle = Entrez.esummary(db="bioproject", id=project_uid)
project_record= Entrez.read(handle)
handle.close()

print(project_record) 

#seleccionamos la informacion especifico 

proj= project_record["DocumentSummarySet"]["DocumentSummary"][0]

#Obtenemos los datos especificos del proyecto 
print(proj.keys()) 

print("\n=== Información del BioProject ===")
print("Tipo de proyecto:", proj["Project_Data_Type"])
print("Metodo del proyecto:", proj["Project_MethodType"])
print("Organismo:", proj["Organism_Name"])
print("Descripción:", proj.get("Project_Description"))

{'DocumentSummarySet': DictElement({'DocumentSummary': [DictElement({'TaxId': '0', 'Project_Id': '552284', 'Project_Acc': 'PRJNA552284', 'Project_Type': 'Primary submission', 'Project_Data_Type': 'Raw sequence reads', 'Sort_By_ProjectType': '563532', 'Sort_By_DataType': '551612', 'Sort_By_Organism': '884981', 'Project_Subtype': '', 'Project_Target_Scope': 'Multispecies', 'Project_Target_Material': 'Genome', 'Project_Target_Capture': 'Whole', 'Project_MethodType': 'Sequencing', 'Project_Method': '', 'Project_Objectives_List': [{'Project_ObjectivesType': 'Raw Sequence Reads', 'Project_Objectives': ''}], 'Registration_Date': '2019/07/02 00:00', 'Project_Name': '', 'Project_Title': 'Babesia divergens strain:Bd Rouen 1987 | cultivar:Babesia divergens human red blood cells culture', 'Project_Description': 'Babesia divergens is a tick-borne, obligate intraerythrocytic apicomplexa parasite responsible of cattle and human babesiosis. While cattle babesiosis produces significant economic annual 

### Consultar con otras bases de datos 

In [ ]:
# funcion elink  

handle= Entrez.elink(dbfrom="bioproject", db="gds", id= project_uid)
linksBioProj= Entrez.read(handle)
handle.close()
# este proyecto no cuenta con links por lo que se hara con otro para ejemplificar 
linksBioProj 


{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['16899'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'PRJNA16899[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'PRJNA16899[All Fields]'}
['16899']


[{'ERROR': [], 'LinkSetDb': [], 'LinkSetDbHistory': [], 'DbFrom': 'bioproject', 'IdList': ['16899']}]

In [ ]:
#Proyecto pasilla 

project_acc = "PRJNA168994"
handle = Entrez.esearch(db="bioproject", term=project_acc)
search_results = Entrez.read(handle)
print(search_results)
handle.close() 

# Extraer solo el UID
project_uid = search_results["IdList"][0]
print("UID del BioProject:", project_uid) 


# Revisar enlaces desde BioProject hacia GEO (GDS), con el uid del bioproject
handle = Entrez.elink(dbfrom="bioproject", db="gds", id=project_uid)
linksBioProj = Entrez.read(handle)
handle.close()

print(linksBioProj)

{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['168994'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'PRJNA168994[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'PRJNA168994[All Fields]'}
UID del BioProject: 168994
[{'ERROR': [], 'LinkSetDb': [{'Link': [{'Id': '200040045'}, {'Id': '200040043'}, {'Id': '200040042'}, {'Id': '200040040'}, {'Id': '200040039'}, {'Id': '200040038'}, {'Id': '200040036'}, {'Id': '200040034'}, {'Id': '200040016'}, {'Id': '200040015'}, {'Id': '200037756'}, {'Id': '200037440'}, {'Id': '200037439'}, {'Id': '200037438'}, {'Id': '200037437'}, {'Id': '200037436'}, {'Id': '200037435'}, {'Id': '200037434'}, {'Id': '200037300'}, {'Id': '200037299'}, {'Id': '200037297'}, {'Id': '200037293'}, {'Id': '200037292'}, {'Id': '200037291'}, {'Id': '200025570'}, {'Id': '200025392'}, {'Id': '200025390'}, {'Id': '200025387'}, {'Id': '200025348'}, {'Id': '200025347'}, {'Id': '200024608'}, {'Id': '200024607'}, {'I

In [8]:
#Obtenemos una lista con links relacionados  
if linksBioProj[0]["LinkSetDb"]:
    geo_ids = [link["Id"] for link in linksBioProj[0]["LinkSetDb"][0]["Link"]]
    print("\nEl proyecto", project_acc, " está asociado a la base de datos de GEO")
    print("UIDs GEO asociados:", geo_ids[:5], "... total:", len(geo_ids))
else:
    print("\nEl proyecto", project_acc, " NO tiene enlaces directos a GEO") 


El proyecto PRJNA168994  está asociado a la base de datos de GEO
UIDs GEO asociados: ['200040045', '200040043', '200040042', '200040040', '200040039'] ... total: 156


### Resumen de un GSE y sus muestras (GSM) 

Al ejecutar esummary sobre un UID de GEO (GSE), obtenemos un diccionario con múltiples campos que describen el experimento. Entre la información más relevante se encuentran:

- Accession: identificador tipo GSE
- title: título del estudio
- summary: descripción general del experimento
- organisms estudiados
- Samples: lista de muestras asociadas (GSM)
- Publicación: fecha de publicación
- Plataforma tecnológica utilizada
- Estos campos permiten comprender rápidamente el alcance y los objetivos del experimento sin necesidad de explorar los registros individuales de cada muestra (GSM). 


In [9]:
# Verificando la información de GEO
geo_uid = geo_ids[1]  # ejemplo, segundo UID
handle = Entrez.esummary(db="gds", id=geo_uid)
summary = Entrez.read(handle)
handle.close()
print(summary[0])
print(summary[0].keys()) 


# imprimiendo los campos
geo_record=summary[0]
print("=== GEO Accession ===")
print("Accession:", geo_record.get("Accession"))
print("Tipo:", geo_record.get("entryType"))
print("Título:", geo_record.get("title"))
print("Resumen:", geo_record.get("summary"))
print("Organismo:", geo_record.get("taxon"))
print("Número de muestras:", geo_record.get("n_samples"))
print("PubMed IDs:", geo_record.get("PubMedIds"))
print("Enlace FTP:", geo_record.get("FTPLink"))
print("Archivos suplementarios:", geo_record.get("suppFile"))
print("Proyectos: ",geo_record.get("Projects"))

# Muestras asociadas
print("\n=== Muestras ===")
for sample in geo_record.get("Samples",[]):
    print(sample['Accession'],":",sample['Title'])

{'Item': [], 'Id': '200040043', 'Accession': 'GSE40043', 'GDS': '', 'title': 'Drosophila melanogaster Pupae 2-4 days', 'summary': 'modENCODE_submission_757 This submission comes from a modENCODE project of Eric Lai. For full list of modENCODE projects, see http://www.genome.gov/26524648 Project Goal: We plan to generate a comprehensive catalog of expressed and functional microRNAs, and generate biological evidence for their regulatory activity. We plan also to delineate the primary transcription units of microRNA genes. Finally, we plan to annotate other classes of non-miRNA expressed small RNAs, as least some of which may define novel classes of small RNA genes. For data usage terms and conditions, please refer to http://www.genome.gov/27528022 and http://www.genome.gov/Pages/Research/ENCODE/ENCODEDataReleasePolicyFinal2008.pdf', 'GPL': '9058', 'GSE': '40043', 'taxon': 'Drosophila melanogaster', 'entryType': 'GSE', 'gdsType': 'Non-coding RNA profiling by high throughput sequencing', '

### Obtener muestras asociadas 
Es hora de obtener los datos de los experimentos que hemos estando consultando 

In [11]:
# Obtener el UID correspondiente a la muestra
gsm_id = "GSM461177"
handle = Entrez.esearch(db="sra", term=gsm_id)
record = Entrez.read(handle)
print(record) 


# Consultar si existe el identificador
if not record["IdList"]:
    print("No se encontraron SRR para", gsm_id)
else:
    sra_id = record["IdList"]
# Obtener la información de la corrida como texto
    handle = Entrez.efetch(db="sra", id=sra_id, rettype="runinfo", retmode="text")
    #este objeto pued contener un objeto binario b'binary_text'
    runinfo = handle.read()
    handle.close()
    
    #si este objeto es de tipo binario hay que convertirlo a texto para poder leerlo 
    if isinstance(runinfo, bytes):
        runinfo = runinfo.decode("utf-8")

# Cargar en pandas el resultado, directamente desde la cadena
    df = pd.read_csv(StringIO(runinfo))
    print(df.columns)
    display(df[['Run','Experiment','Platform','LibraryName','LibraryLayout','Sample','ScientificName','SampleName']].head())

{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['14790'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'GSM461177[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'GSM461177[All Fields]'}
Index(['Run', 'ReleaseDate', 'LoadDate', 'spots', 'bases', 'spots_with_mates',
       'avgLength', 'size_MB', 'AssemblyName', 'download_path', 'Experiment',
       'LibraryName', 'LibraryStrategy', 'LibrarySelection', 'LibrarySource',
       'LibraryLayout', 'InsertSize', 'InsertDev', 'Platform', 'Model',
       'SRAStudy', 'BioProject', 'Study_Pubmed_id', 'ProjectID', 'Sample',
       'BioSample', 'SampleType', 'TaxID', 'ScientificName', 'SampleName',
       'g1k_pop_code', 'source', 'g1k_analysis_group', 'Subject_ID', 'Sex',
       'Disease', 'Tumor', 'Affection_Status', 'Analyte_Type',
       'Histological_Type', 'Body_Site', 'CenterName', 'Submission',
       'dbgap_study_accession', 'Consent', 'RunHash', 'ReadHash'],
      dtype='o

,Run,Experiment,Platform,LibraryName,LibraryLayout,Sample,ScientificName,SampleName
0,SRR031714,SRX014459,ILLUMINA,S2_DRSC_Untreated-3,PAIRED,SRS008447,Drosophila melanogaster,GSM461177
1,SRR031715,SRX014459,ILLUMINA,S2_DRSC_Untreated-3,PAIRED,SRS008447,Drosophila melanogaster,GSM461177


Ahora ya sabemos los archivos SRR asosciados a una muestra de un proyecto sigue la descarga de los datos 

### Descarga con SRA ToolKit 

El SRA Toolkit (sra-tools) es un conjunto de programas creados por el NCBI para descargar y manejar datos de secuenciación masiva almacenados en el SRA (Sequence Read Archive).
Este toolkit incluye diversas funciones que permiten explorar, validar, convertir y descargar datos.

En este curso nos enfocaremos únicamente en las funciones de descarga y conversión, necesarias para obtener las secuencias en formato FASTQ para nuestro análisis. No obstante, el toolkit cuenta con otras utilidades que pueden consultarse en la documentación oficial o ejecutando --help en cualquier comando.

Funiones de descarga y conversión

prefetch: descarga los archivos SRR completos desde el servidor del SRA hacia tu disco local.
fastq-dump / fasterq-dump: extrae las secuencias en formato FASTQ desde los archivos SRR descargados. Permite separar lecturas pareadas (--split-files) y comprimir los resultados (--gzip).
Con esto, podemos explorar los SRR asociados a cada muestra, conocer la plataforma, el tipo de librería y otros metadatos relevantes para análisis de RNA-Seq. 


#### subprocces
Las herramientas de SRA Toolkit se ejecutan normalmente desde la línea de comandos en una terminal. Como no forman parte de Python ni de Biopython, para integrarlas dentro de un script usamos la biblioteca estándar subprocess, que permite ejecutar programas externos y capturar su salida. Esto facilita automatizar la descarga y conversión de múltiples archivos SRR en un flujo de análisis reproducible. 


Función	
- subprocess.run()	Ejecuta un comando externo y espera a que termine, tiene una lista como parametros que es la sintaxis del comando seaprada por partes 
```
args, check, stdout, stderr, cwd, text


```
`subprocess.run(["ls","-l"], check=True, stdout=subprocess.PIPE, text=True)`

- subprocess.Popen()	Ejecuta un proceso y permite interactuar con él	

```
stdin, stdout, stderr, text, cwd

```	

`p = subprocess.Popen(["echo","Hola"], stdout=subprocess.PIPE, text=True); output, errors = p.communicate()`

- subprocess.call()	Ejecuta un comando y devuelve solo el código de salida	
```
args, cwd
```	

`ret_code = subprocess.call(["ls","-l"])`

- subprocess.check_output()	Ejecuta un comando y devuelve la salida como string	args, stderr, text	output = subprocess.check_output(["ls","-l"], text=True)

In [13]:
# Crear la ruta del directorio de entrada y salida de datos
output_dir = "C://Users//vrahu//OneDrive//Documentos//Ismadlsh//CCG//2026-S3//bioPython//babesia_genomic_analysis_wrk//data"

# Hacer el prefetch de los archivos srr
subprocess.run(["prefetch", "--output-directory", output_dir, "SRR031714"], check=True)
subprocess.run(["prefetch", "--output-directory", output_dir, "SRR031715"], check=True)

FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado

En este fragmento de código definimos primero el directorio donde se guardarán los datos (output_dir). Luego utilizamos la función subprocess.run() para ejecutar el comando prefetch del SRA Toolkit, indicando el identificador de cada archivo (SRR031714 y SRR031715) junto con la opción --output-directory, que especifica la ruta de destino. La opción check=True asegura que, si ocurre un error en la ejecución del comando, Python detenga el script y muestre el error.

Una vez descargados los archivos SRR, el siguiente paso es convertirlos al formato FASTQ, que es el estándar para trabajar con secuencias crudas. El siguiente código realiza la conversión para los archivos SRR031714 y SRR031715, utilizando la instrucción fastq-dump.

In [ ]:
# Obtener las secuencias en formato fastq

#obtenemos la direccion del archivo 
srr_output_dir = os.path.join(output_dir, "SRR031714")
print(srr_output_dir)
#usamos el comando fastdump para obetmer lps archivps fastq 
subprocess.run(["fastq-dump", "SRR031714", "-O", srr_output_dir,"--split-files", "--gzip"], check=True)
srr_output_dir = os.path.join(output_dir, "SRR031715")
print(srr_output_dir)
subprocess.run(["fastq-dump", "SRR031715", "-O", srr_output_dir,"--split-files", "--gzip"], check=True)